In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import files
myfile = files.upload()

Saving 2data.csv to 2data.csv


In [3]:
from google.colab import files
myfile = files.upload()

Saving fromday2_F.csv to fromday2_F.csv
Saving fromday2_M.csv to fromday2_M.csv
Saving fromday2_R.csv to fromday2_R.csv


In [4]:
R = pd.read_csv("fromday2_R.csv")
F = pd.read_csv("fromday2_F.csv")
M = pd.read_csv("fromday2_M.csv")
D = pd.read_csv("2data.csv")

In [5]:
print(R.head())
print(R.shape)

                    주문자ID번호  R점수
0    6773359@n010-9381-5574    5
1   49938077@n010-8288-1034    5
2       plom90010-4209-3577    5
3             010-2926-1299    5
4  dlagmltmd95010-3205-7859    5
(4774, 2)


In [7]:
RF = pd.merge(R, F, on="주문자ID번호")
RFM = pd.merge(RF,M, on="주문자ID번호")
RFMD = pd.merge(RFM,D, on="주문자ID번호")

In [8]:
RFMD

,주문자ID번호,R점수,F점수,M점수,R : 최근주문일,F : 주문수,M : 총 금액
0,6773359@n010-9381-5574,5,3,4,2021-07-28 23:48,16.0,1778800.0
1,49938077@n010-8288-1034,5,3,3,2021-07-28 23:37,7.0,487100.0
2,plom90010-4209-3577,5,4,4,2021-07-28 23:01,33.0,3186820.0
3,010-2926-1299,5,3,1,2021-07-28 23:00,11.0,40400.0
4,dlagmltmd95010-3205-7859,5,2,2,2021-07-28 22:56,3.0,136200.0
...,...,...,...,...,...,...,...
4769,010-7414-4920,1,2,2,2021-01-29 7:34,3.0,288000.0
4770,755214608592690@f010-4099-9220,1,2,1,2021-01-29 1:13,2.0,35400.0
4771,1601304767@k010-2316-3350,1,2,2,2021-01-29 0:43,3.0,150000.0
4772,정혜빈jhb8****,1,1,1,2021/01/29 00:42:42,1.0,25500.0


In [9]:
import pandas as pd
import numpy as np
 
from tqdm import tqdm


In [14]:
def get_score(level, data, reverse = False):
    '''
    Description :
    level안에 있는 원소를 기준으로
    1 ~ len(level)+ 1 까지 점수를 부여하는 함수
    
    Parameters :
    level = 튜플 또는 리스트 타입의 숫자형 데이터이며 반드시 오름차순으로 정렬되어 있어야함.
    예 - [1,2,3,4,5] O, [5,4,3,2,1] X, [1,3,2,10,4] X 
    data = 점수를 부여할 데이터. 순회가능한(iterable) 데이터 형식
    reverse = 점수가 높을 때 그에 해당하는 값을 낮게 설정하고 싶을 때 True
    return :
    점수를 담고 있는 리스트 반환
    '''
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue
    if reverse:
        return [len(level)+2-x for x in score]
    else:
        return score 
 
grid_number = 100 ## 눈금 개수, 너무 크게 잡으면 메모리 문제가 발생할 수 있음.
weights = []
for j in range(grid_number+1):
    weights += [(i/grid_number,j/grid_number,(grid_number-i-j)/grid_number)
                  for i in range(grid_number+1-j)]
num_class = 5 ## 클래스 개수
class_level = np.linspace(1,5,num_class+1)[1:-1] ## 클래스를 나누는 지점을 정한다.
total_amount_of_sales = RFMD['M : 총 금액'].sum() ## 구매금액 총합 = 총 매출


In [16]:

print(class_level)

[1.8 2.6 3.4 4.2]


In [18]:
max_std = 0 ## 표준편차 초기값
for w in tqdm(weights,position=0,desc = '[Finding Optimal weights]'):
    ## 주어진 가중치에 따른 고객별 점수 계산
    score = w[0]*RFMD['R점수'] + \
                        w[1]*RFMD['F점수'] + \
                        w[2]*RFMD['M점수'] 
    RFMD['Class'] = get_score(class_level,score,True) ## 점수를 이용하여 고객별 등급 부여
    ## 등급별로 구매금액을 집계한다.
    grouped_RFMD = RFMD.groupby('Class')['M : 총 금액'].sum().reset_index()
        
    ## 클래스별 구매금액을 총구매금액으로 나누어 클래스별 매출 기여도 계산
    grouped_RFMD['M : 총 금액'] = grouped_RFMD['M : 총 금액'].map(lambda x : x/total_amount_of_sales)
    std_sales = grouped_RFMD['M : 총 금액'].std() ## 매출 기여도의 표준편차 계산
    if max_std <= std_sales:
        max_std = std_sales ## 표준편차 최대값 업데이트
        optimal_weights = w  ## 가중치 업데이트

[Finding Optimal weights]: 100%|██████████| 5151/5151 [08:28<00:00, 10.12it/s]


In [19]:
print(optimal_weights)

(0.74, 0.26, 0.0)
